In [1]:
from pathlib import Path
import subprocess, sys, time
import shutil

In [2]:
MULTIPROBE = True # This determines if TPrime is used to align multiple streams

In [3]:
dt_folder = Path(r"C:\Users\Data Analysis\Desktop\project_trainingAggression\Data")

ni_data = sorted(dt_folder.rglob("*nidq.bin"))
run_folders = [rf.parent for rf in ni_data]
s = "catgt_"
catGT_runs = [cgt.name.split(s)[1] for cgt in sorted(dt_folder.rglob("catgt*"))]

unprocessed_neuralData_folders = [rf for rf in run_folders if rf.name[len(s):] not in catGT_runs]

if len(unprocessed_neuralData_folders) == 0:
    print("No files to process found.")
else: 
    print("Runs to process with catGT:")
    for rf in unprocessed_neuralData_folders:
        print("  ", rf)

Runs to process with catGT:
   C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251007_mouse1010826\Day01\neuralData\20251007_m1010826_obs1_g0
   C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251007_mouse1010826\Day03\neuralData\20251007_m1010826_obs2_g0
   C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251007_mouse1010826\Day08\neuralData\20251007_m1010826_obs3_g0
   C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251007_mouse1010826\Day10\neuralData\20251007_m1010826_obs4_g0
   C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251007_mouse1010826\Day12\neuralData\20251007_m1010826_obs5_g0


# **Pre-processing**

https://billkarsh.github.io/SpikeGLX/help/syncEdges/Sync_edges/

## catGT: Single probe demultiplexing corrections and TTL event extraction

**Documentation:** file:///C:/Users/Data%20Analysis/Desktop/CatGT-win/ReadMe.html

If Exit code 42, see CatGT log in \project_trainingAggression\Code\CatGT.txt

In [4]:
catGT = (r"C:\Users\Data Analysis\Desktop\CatGT-win\CatGT.exe")

for folderRun in unprocessed_neuralData_folders:

    start = time.time()

    dataFolder = folderRun.parent
    spikeGLX_run = folderRun.name 
    if ("_g0" in spikeGLX_run):
        spikeGLX_run = spikeGLX_run[0:-3]# Removes "_g0" from filename

    print(f"Multiplexer temporal shift correction in session: {spikeGLX_run}")

    cmd = [
            str(catGT),
            f"-dir={dataFolder}",  
            f"-run={spikeGLX_run}",
            "-g=0",
            "-t=0",            
            "-ap",              # Process Action Potential (.ap.bin) files
            "-prb=0:2",         # Probe 1 to 3
            "-ni",              # Process the auxiliary data (.niqd.bin) containing both Analog and Digital channels
            "-prb_fld",         # 1 probe per folder
            "-maxsecs=1205.00", # 20 min + 5 seconds


            # ========= -xd = js,ip,word,bit,pulsewidth(ms),tol(ms) =========

            "-xid=0,0,3,2,50,5",               # Sweetened milk events: falling edges (P.02)
            #"-xd=0,0,8,2,15000, 1000",        # Sweetened milk events: rising edges (P.02)

            "-xd=0,0,3,3,0",                   # LED red light ON: rising edges (P.03)
            "-xid=0,0,3,3,360000,5000",        # LED white light ON: falling edge (P.03)  

            "-xd=0,0,3,4, 1000000,500000",     # Gate OPEN: rising edge (P.04)

            "-xd=0,0,3,5,3500,500",            # uArm ON: rising edges (P.05)
            "-xid=0,0,3,5,11000,3000",         # uArm OFF: falling edges (P.05) (Not all of them will be picked up but it is good to know the average trial duration)

            f"-dest={dataFolder}",
            "-pass1_force_ni_ob_bin",
            "-out_prb_fld",
          ]
    
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in p.stdout:
        sys.stdout.write(line)
    rc = p.wait()

    shutil.rmtree(folderRun) # Deleting original run folder

    end = time.time()

    print(f"Exit code: {rc}. Running time (1 SpikeGLX run): {(end - start)/60: .2f} min.\n")


Multiplexer temporal shift correction in session: 20251007_m1010826_obs1
Exit code: 0. Running time (1 SpikeGLX run):  29.46 min.

Multiplexer temporal shift correction in session: 20251007_m1010826_obs2
Exit code: 0. Running time (1 SpikeGLX run):  29.73 min.

Multiplexer temporal shift correction in session: 20251007_m1010826_obs3
Exit code: 0. Running time (1 SpikeGLX run):  33.77 min.

Multiplexer temporal shift correction in session: 20251007_m1010826_obs4
Exit code: 0. Running time (1 SpikeGLX run):  31.11 min.

Multiplexer temporal shift correction in session: 20251007_m1010826_obs5
Exit code: 0. Running time (1 SpikeGLX run):  32.35 min.



## kilosort4: Spike sorting after High-Pass FIR filter (300 Hz), Common Average Referencing (CAR) and whitening

**Documentation:** https://kilosort.readthedocs.io/en/latest/

In [10]:
def filter_unprocessed(files):
    """Keep only those whose neuralData parent folder has no kilosort4/ directory."""
    out = []
    for f in files:
        kilosort_out = f.parent / "kilosort4"
        if not kilosort_out.exists():
            out.append(f)
    return out

def read_spikeglx_meta(meta_path: Path) -> dict:
    """Parse a SpikeGLX .meta into a dict."""
    metafile = {}
    with meta_path.open('r', encoding='utf-8') as f:
        for line in f:
            if '=' in line:
                k, v = line.strip().split('=', 1)
                metafile[k] = v
    return metafile

tcat_bin_files  = filter_unprocessed(sorted(dt_folder.rglob("*_tcat.imec*.ap.bin")))
tcat_meta_files = filter_unprocessed(sorted(dt_folder.rglob("*_tcat.imec*.ap.meta")))

if len(tcat_bin_files) == 0:
    print(" No files to process found.")
else:
    print("Runs to spike sort:")
    for rf in tcat_bin_files:
        print("  ", rf)

Runs to spike sort:
   C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20250817_mouse975826\Day01\neuralData\catgt_20250817_m975826_obs1_g0\20250817_m975826_obs1_g0_imec0\20250817_m975826_obs1_g0_tcat.imec0.ap.bin
   C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20250817_mouse975826\Day01\neuralData\catgt_20250817_m975826_obs1_g0\20250817_m975826_obs1_g0_imec1\20250817_m975826_obs1_g0_tcat.imec1.ap.bin
   C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20250817_mouse975826\Day01\neuralData\catgt_20250817_m975826_obs1_g0\20250817_m975826_obs1_g0_imec2\20250817_m975826_obs1_g0_tcat.imec2.ap.bin


**Kilosort4 output files description:** https://kilosort.readthedocs.io/en/stable/export_files.html

In [ ]:
import kilosort
import torch

for it, probe_run in enumerate(tcat_bin_files):

    start = time.time()

    print(f"\n ============= Running kilosort4 on {probe_run.name} ============= ")

    metafile = read_spikeglx_meta(tcat_meta_files[it])

    settings = {
                'filename': probe_run,
                'n_chan_bin': 385,                                      # 384 + 1 because synch channel is included
                'fs': float(metafile["imSampRate"]),                    # Each headstage has a slight different clock/sampling rate. This is was obtained through the synch channel after a 40 min calibration run on SpikeGLX 
                'highpass_cutoff': 300.0              
            }

    ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = kilosort.run_kilosort(
                                                                                        settings=settings,
                                                                                        do_CAR = True,                                         # Common Average Referencing
                                                                                        probe_name = "neuropixPhase3B2_kilosortChanMap.mat",   # Neuropixels 1.0 channel map 
                                                                                        data_dtype = 'int16',                                   
                                                                                        #save_preprocessed_copy = True,                         # Saves a copy pre-processed copy of the data before spike sorting
                                                                                        results_dir = f"{tcat_meta_files[it].parent}/kilosort4",              
                                                                                        device=torch.device('cuda')              
                                                                                        )
    
    end = time.time()
    print(f"\nRunning time (1 probe): {(end - start)/60: .2f} min.\n")

    print(" ================================================================\n")



kilosort.run_kilosort: Kilosort version 4.1.1
kilosort.run_kilosort: Python version 3.12.7
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: System information:
kilosort.run_kilosort: Windows-11-10.0.26100-SP0 AMD64
kilosort.run_kilosort: Intel64 Family 6 Model 85 Stepping 7, GenuineIntel
kilosort.run_kilosort: Using CUDA device: NVIDIA GeForce RTX 3060 12.00GB
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Sorting [WindowsPath('C:/Users/Data Analysis/Desktop/project_trainingAggression/Data/20251006 _mouse1010820/Day01/neuralData/catgt_20251006_m1010820_obs1_g0/20251006_m1010820_obs1_g0_imec0/20251006_m1010820_obs1_g0_tcat.imec0.ap.bin')]
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage before sorting
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:     6.30 %
kilosort.run_kilosort: Mem used:     16.50 %     |      21.18 GB
kilosort


 ============= Running kilosort4 on 20251006_m1010820_obs1_g0_tcat.imec0.ap.bin ============= 


kilosort.run_kilosort: Preprocessing filters computed in 6.55s; total 6.57s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:    27.80 %
kilosort.run_kilosort: Mem used:     16.50 %     |      21.18 GB
kilosort.run_kilosort: Mem avail:    106.82 / 128.00 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:   44.62 %     |      5.35   /    12.00 GB
kilosort.run_kilosort: Allocated:     0.09 %     |      0.01   /    12.00 GB
kilosort.run_kilosort: Max alloc:    10.89 %     |      1.31   /    12.00 GB
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing drift correction.
kilosort.run_kilosort: ---------------------------------


Running time (1 probe):  21.81 min.



 ============= Running kilosort4 on 20251006_m1010820_obs1_g0_tcat.imec1.ap.bin ============= 


kilosort.run_kilosort: Preprocessing filters computed in 4.36s; total 4.42s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:    16.60 %
kilosort.run_kilosort: Mem used:     16.20 %     |      20.79 GB
kilosort.run_kilosort: Mem avail:    107.21 / 128.00 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:   44.62 %     |      5.35   /    12.00 GB
kilosort.run_kilosort: Allocated:     0.09 %     |      0.01   /    12.00 GB
kilosort.run_kilosort: Max alloc:    10.89 %     |      1.31   /    12.00 GB
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing drift correction.
kilosort.run_kilosort: ---------------------------------


Running time (1 probe):  25.94 min.



 ============= Running kilosort4 on 20251006_m1010820_obs1_g0_tcat.imec2.ap.bin ============= 


kilosort.run_kilosort: Preprocessing filters computed in 2.82s; total 2.87s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:    24.20 %
kilosort.run_kilosort: Mem used:     15.50 %     |      19.89 GB
kilosort.run_kilosort: Mem avail:    108.11 / 128.00 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:   44.62 %     |      5.35   /    12.00 GB
kilosort.run_kilosort: Allocated:     0.09 %     |      0.01   /    12.00 GB
kilosort.run_kilosort: Max alloc:    10.89 %     |      1.31   /    12.00 GB
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing drift correction.
kilosort.run_kilosort: ---------------------------------


Running time (1 probe):  23.57 min.




## TPrime: Align events across probes and NI analog + TTL data

In [22]:
import numpy as np

# First a conversion from samples to seconds is required in the spike sorted data

spk_time_files = dt_folder.rglob("*spike_times.npy")
aready_processed = [ard_prc.parent for ard_prc in (dt_folder.rglob("*spike_seconds.npy"))]

spk_time_files_2proc = [spk_file for spk_file in spk_time_files if spk_file.parent not in aready_processed]

for spk_file in spk_time_files_2proc:

    probe_folder = spk_file.parent.parent

    print(spk_file, probe_folder)

    metafile = read_spikeglx_meta(next(probe_folder.glob("*.meta")))

    fl = np.load(spk_file)

    samples_to_seconds = fl/float(metafile["imSampRate"])
    newfile_path = spk_file.parent / "spike_seconds.npy"

    np.save(newfile_path, samples_to_seconds)

C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251006 _mouse1010820\Day01\neuralData\catgt_20251006_m1010820_obs1_g0\20251006_m1010820_obs1_g0_imec0\kilosort4\spike_times.npy C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251006 _mouse1010820\Day01\neuralData\catgt_20251006_m1010820_obs1_g0\20251006_m1010820_obs1_g0_imec0
C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251006 _mouse1010820\Day01\neuralData\catgt_20251006_m1010820_obs1_g0\20251006_m1010820_obs1_g0_imec1\kilosort4\spike_times.npy C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251006 _mouse1010820\Day01\neuralData\catgt_20251006_m1010820_obs1_g0\20251006_m1010820_obs1_g0_imec1
C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251006 _mouse1010820\Day01\neuralData\catgt_20251006_m1010820_obs1_g0\20251006_m1010820_obs1_g0_imec2\kilosort4\spike_times.npy C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251006 _mouse1010820\D

In [23]:
if MULTIPROBE:
    
    tprime = (r"C:\Users\Data Analysis\Desktop\TPrime-win\TPrime.exe")

    already_converted = [stream.parent.parent.parent for stream in dt_folder.rglob("*spike_seconds_prb*_adj.npy")]

    all_master_streams_ni = dt_folder.rglob("*nidq.xd_3_0_500.txt")
    master_streams_ni_2proc = [stream for stream in all_master_streams_ni if stream.parent not in already_converted]

    for id, ni_stream in enumerate(master_streams_ni_2proc):

        start = time.time()
        
        print(f"Aligning session: {ni_stream.parent}")

        prb_streams = [prb for prb in (ni_stream.parent.rglob("*tcat.imec*.ap.xd*"))]

        spk_sec_prb0 = next(prb_streams[0].parent.rglob("*spike_seconds.npy"))
        conv_spk_sec_prb0 = spk_sec_prb0.parent / "spike_seconds_prb0_adj.npy"

        spk_sec_prb1 = next(prb_streams[1].parent.rglob("*spike_seconds.npy"))
        conv_spk_sec_prb1 = spk_sec_prb1.parent / "spike_seconds_prb1_adj.npy"

        spk_sec_prb2 = next(prb_streams[2].parent.rglob("*spike_seconds.npy"))
        conv_spk_sec_prb2 = spk_sec_prb2.parent / "spike_seconds_prb2_adj.npy"

        cmd = [
        tprime,
        "-syncperiod=1.0",
        f"-tostream={ni_stream}",
        f"-fromstream=1,{prb_streams[0]}",
        f"-fromstream=2,{prb_streams[1]}",
        f"-fromstream=3,{prb_streams[2]}",
        f"-events=1,{spk_sec_prb0},{conv_spk_sec_prb0}",
        f"-events=2,{spk_sec_prb1},{conv_spk_sec_prb1}",
        f"-events=3,{spk_sec_prb2},{conv_spk_sec_prb2}",
        ]

        start = time.time()

        p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout:
            sys.stdout.write(line)
        rc = p.wait()

        end = time.time()

        print(f"Exit code: {rc}. Running time (1 SpikeGLX run): {(end - start): .2f} sec.\n")

Aligning session: C:\Users\Data Analysis\Desktop\project_trainingAggression\Data\20251006 _mouse1010820\Day01\neuralData\catgt_20251006_m1010820_obs1_g0
Exit code: 0. Running time (1 SpikeGLX run):  0.55 sec.

